# RNNs

Y'all punks can get the data from kaggle here: https://www.kaggle.com/uciml/sms-spam-collection-dataset/kernels 

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/spam.csv', encoding = 'latin1')

In [2]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
from html import parser

# Drop the empty columns
df = df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)

# Rename the columns
df = df.rename(index=str, columns={"v1": "classification", "v2": "message"})

# Remove HTML encodings
df['message'] = df['message'].apply(parser.unescape)
df.head()

,classification,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df['classification'] = pd.get_dummies(df['classification'])

In [5]:
print(df.head())

   classification                                            message
0               1  Go until jurong point, crazy.. Available only ...
1               1                      Ok lar... Joking wif u oni...
2               0  Free entry in 2 a wkly comp to win FA Cup fina...
3               1  U dun say so early hor... U c already then say...
4               1  Nah I don't think he goes to usf, he lives aro...


In [6]:
y = np.array(df['classification'].values.astype(np.int))
np.unique(y)

array([0, 1])

In [28]:
import keras


y = np.array(df['classification'].values)
X = df['message'].values
#print(X)

#y = [0 if y_ == "spam" else 1 for y_ in y]
y_ohe = keras.utils.to_categorical(y)
y_ohe


array([[ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

We need to find how long our longest message is

In [8]:
max = 0
count = 0
for i in X:
    if len(i) > max:
        #if len(i) < 200:
        max = len(i)
        #else:
            #count +=1
        
print(max)
print(count)

910
0


In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
NUM_TOP_WORDS = None

tokenizer = Tokenizer(num_words=NUM_TOP_WORDS)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(X)
sequences = pad_sequences(sequences)
MAX_TEXT_LEN = len(sequences[0])
print(MAX_TEXT_LEN)

189


In [99]:
import numpy as np
from keras.datasets import imdb
from sklearn.model_selection import train_test_split

y = y.astype(bool)

X_train, X_test, y_train, y_test = train_test_split(
    sequences, y, test_size=0.2, stratify=y, random_state=42)


# truncate and pad input sequences to be the same length
max_text_length = 189

NUM_CLASSES = 1

In [12]:
EMBED_SIZE = 100
# the embed size should match the file you load glove from
embeddings_index = {}
f = open('data/glove.6B/glove.6B.100d.txt', encoding="utf8")
# save key/array pairs of the embeddings
#  the key of the dictionary is the word, the array is the embedding
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# now fill in the matrix, using the ordering from the
#  keras word tokenizer from before
embedding_matrix = np.zeros((len(word_index) + 1, EMBED_SIZE))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

Found 400000 word vectors.
(8920, 100)


In [13]:
from keras.layers import Embedding



# GRU RNN

In [134]:
from keras.models import Input, Model
from keras.layers import Dense, GRU, Activation
from keras.regularizers import l2

l2_lambda = 0.0001

input_holder = Input(shape=(189,))
embedding_layer = Embedding(len(word_index) + 1,
                            EMBED_SIZE,
                            weights=[embedding_matrix],
                            input_length=MAX_TEXT_LEN,
                            trainable=False)(input_holder)

gru_layer = GRU(100,dropout=0.2, recurrent_dropout=0.2)(embedding_layer)
dense1 = Dense(NUM_CLASSES, 
                activation='sigmoid',
                kernel_initializer='glorot_uniform',
                kernel_regularizer=l2(l2_lambda)
             )(gru_layer)

rnn_gru = Model(inputs=input_holder, outputs=dense1)
rnn_gru.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
print(rnn_gru.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 189)               0         
_________________________________________________________________
embedding_22 (Embedding)     (None, 189, 100)          892000    
_________________________________________________________________
gru_2 (GRU)                  (None, 100)               60300     
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 101       
Total params: 952,401
Trainable params: 60,401
Non-trainable params: 892,000
_________________________________________________________________
None


In [135]:
rnn_gru.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Train on 4457 samples, validate on 1115 samples
Epoch 1/3
4457/4457 [==============================] - 18s - loss: 0.2159 - acc: 0.9096 - val_loss: 0.1078 - val_acc: 0.9623
Epoch 2/3
4457/4457 [==============================] - 16s - loss: 0.1011 - acc: 0.9650 - val_loss: 0.0882 - val_acc: 0.9722
Epoch 3/3
4457/4457 [==============================] - 16s - loss: 0.0842 - acc: 0.9731 - val_loss: 0.0798 - val_acc: 0.9758


In [137]:
yhat_gru = np.round(rnn_gru.predict(X_test, verbose=1))

1115/1115 [==============================] - 2s     


In [138]:
from sklearn import metrics as mt
rec = mt.recall_score(y_test,yhat_gru)
acc = mt.accuracy_score(y_test,yhat_gru)
print(rec)
print(acc)

0.996894409938
0.975784753363


# LSTM RNN

In [129]:
from keras.models import Input, Model
from keras.layers import Dense, LSTM, Activation
from keras.regularizers import l2

l2_lambda = 0.0001

input_holder = Input(shape=(189,))
embedding_layer = Embedding(len(word_index) + 1,
                            EMBED_SIZE,
                            weights=[embedding_matrix],
                            input_length=MAX_TEXT_LEN,
                            trainable=False)(input_holder)

lstm_layer = LSTM(100,dropout=0.2, recurrent_dropout=0.2)(embedding_layer)
dense1 = Dense(NUM_CLASSES, 
                activation='sigmoid',
                kernel_initializer='glorot_uniform',
                kernel_regularizer=l2(l2_lambda)
             )(lstm_layer)

rnn_lstm = Model(inputs=input_holder, outputs=dense1)
rnn_lstm.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
print(rnn_lstm.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 189)               0         
_________________________________________________________________
embedding_21 (Embedding)     (None, 189, 100)          892000    
_________________________________________________________________
lstm_21 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 101       
Total params: 972,501
Trainable params: 80,501
Non-trainable params: 892,000
_________________________________________________________________
None


In [130]:
rnn_lstm.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Train on 4457 samples, validate on 1115 samples
Epoch 1/3
4457/4457 [==============================] - 22s - loss: 0.1926 - acc: 0.9242 - val_loss: 0.1238 - val_acc: 0.9614
Epoch 2/3
4457/4457 [==============================] - 20s - loss: 0.1071 - acc: 0.9628 - val_loss: 0.0883 - val_acc: 0.9740
Epoch 3/3
4457/4457 [==============================] - 20s - loss: 0.0908 - acc: 0.9686 - val_loss: 0.0833 - val_acc: 0.9722


In [131]:
yhat = np.round(rnn_lstm.predict(X_test, verbose=1))
#yhat = np.array([[0,1] if np.argmax(x) == 1 else [1,0] for x in yhat]).astype(float)
#yhat = [list(x).index(1.0) for x in list(yhat)]

#y_test = np.array([[0,1] if np.argmax(x) == 1 else [1,0] for x in y_test]).astype(float)
#y_test = [list(x).index(1.0) for x in list(y_test)]

1115/1115 [==============================] - 3s     


In [132]:
from sklearn import metrics as mt
rec = mt.recall_score(y_test,yhat)
acc = mt.accuracy_score(y_test,yhat)
print(rec)
print(acc)

0.993788819876
0.972197309417


In [44]:
print(np.array(y_train))

[1 1 1 ..., 1 1 1]


In [41]:
print(yhat)

[[ 0.99228644]
 [ 0.99404907]
 [ 0.990049  ]
 ..., 
 [ 0.99001336]
 [ 0.99323457]
 [ 0.99784422]]


In [96]:
type(yhat[0][0])

numpy.float32

In [104]:
type(y_test[0])

numpy.bool_

## GridSearch

In [ ]:
def gs_make_lstm(l2_lambda=0.0001, dropout=0.2, recurrent_dropout=0.2, activation='sigmoid', kernel_initializer='glorot_uniform'):
    input_holder = Input(shape=(189,))
    embedding_layer = Embedding(len(word_index) + 1,
                                EMBED_SIZE,
                                weights=[embedding_matrix],
                                input_length=MAX_TEXT_LEN,
                                trainable=False)(input_holder)

    lstm_layer = LSTM(100,dropout=dropout, recurrent_dropout=recurrent_dropout)(embedding_layer)
    dense1 = Dense(NUM_CLASSES, 
                    activation=activation,
                    kernel_initializer=kernel_initializer,
                    kernel_regularizer=l2(l2_lambda)
                 )(lstm_layer)

    rnn_lstm_gs = Model(inputs=input_holder, outputs=dense1)
    rnn_lstm_gs.compile(loss='binary_crossentropy', 
                  optimizer='rmsprop', 
                  metrics=['accuracy'])
    return rnn_lstm_gs

def gs_make_gru(l2_lambda=0.0001, dropout=0.2, recurrent_dropout=0.2, activation='sigmoid', kernel_initializer='glorot_uniform'):
    input_holder = Input(shape=(189,))
    embedding_layer = Embedding(len(word_index) + 1,
                                EMBED_SIZE,
                                weights=[embedding_matrix],
                                input_length=MAX_TEXT_LEN,
                                trainable=False)(input_holder)

    gru_layer = GRU(100,dropout=dropout, recurrent_dropout=recurrent_dropout)(embedding_layer)
    dense1 = Dense(NUM_CLASSES, 
                    activation=activation,
                    kernel_initializer=kernel_initializer,
                    kernel_regularizer=l2(l2_lambda)
                 )(gru_layer)

    rnn_gru = Model(inputs=input_holder, outputs=dense1)
    rnn_gru.compile(loss='binary_crossentropy', 
                  optimizer='rmsprop', 
                  metrics=['accuracy'])
    print(rnn_gru.summary())

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
#LSTM
model = KerasClassifier(build_fn=gs_make_lstm)
param_grid = {'epochs'=[2,4,6], 'l2_lambda'=[0.000001,0.0001,0.01], 'dropout'=[0.1,0.2,0.3], 'recurrent_dropout'=[0.1,0.2,0.3], 'activation'=['sigmoid', 'relu'], 'kernel_initializer'=['glorot_uniform','he_normal']}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train, validation_data=(X_test, y_test))

In [ ]:
#GRU
model = KerasClassifier(build_fn=gs_make_gru)
param_grid = {'epochs'=[2,4,6], 'l2_lambda'=[0.000001,0.0001,0.01], 'dropout'=[0.1,0.2,0.3], 'recurrent_dropout'=[0.1,0.2,0.3], 'activation'=['sigmoid', 'relu'], 'kernel_initializer'=['glorot_uniform','he_normal']}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train, validation_data=(X_test, y_test))